In [1]:
from metrics.knowmem import eval as eval_knowmem
from metrics.knowmem import get_prefix_before_words_occur
from utils import load_model, load_tokenizer, write_csv, read_json, write_json, load_csv
from constants import SUPPORTED_METRICS, CORPORA, LLAMA_DIR, DEFAULT_DATA, AUC_RETRAIN

import os
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
from typing import List, Dict, Literal
from pandas import DataFrame
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_dir = LLAMA_DIR
model_dir = 'muse-bench/MUSE-Books_target'
model_dir = LLAMA_DIR
# tokenizer_dir = "EleutherAI/pythia-6.9b"
# model_dir = "EleutherAI/pythia-6.9b"
model = load_model(model_dir).to(device)
tokenizer = load_tokenizer(tokenizer_dir)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
# knowmem_retain_qa_icl_file = DEFAULT_DATA['books']
knowmem_forget_qa_icl_file = "data/books/knowmem/forget_qa_icl.json"

questions = ["Who is the author of the Fourth Wing?", 
             "What is the capital of France?", 
             "Who wrote 'To Kill a Mockingbird'?", 
             "What is the largest planet in our solar system?",
             "who is Harry Potter's best friend?",
             "what is the name of the school Harry Potter goes to?",
             "who is the author of the Harry Potter series?",
             "Dumbledore is the headmaster of which school?"]


# data = load_csv(knowmem_retain_qa_file)

icl = read_json(knowmem_forget_qa_icl_file)
icl_qs=[d['question'] for d in icl]
icl_as=[d['answer'] for d in icl]

answers_icl = []
general_prompt: str = ""

# Few-shot prompting
for question, answer in zip(icl_qs, icl_as):
    general_prompt += f"Question: {question}\nAnswer: {answer}\n\n"

for question in questions:
    prompt = general_prompt + f"Question: {question}\nAnswer: "

    # Encode the `prompt` into `input_ids`
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        add_special_tokens=True).input_ids

    # Use the `model` to generate the continuation of the `input_ids`.
    output_ids = model.generate(
        input_ids.to(model.device),
        max_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id)
    output_ids = output_ids[:, len(input_ids[0]):]

    output = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True)[0]

    stripped_output = get_prefix_before_words_occur(output, ["\n\n", "\nQuestion", "Question:"])
    answers_icl.append(stripped_output.strip())

    # answers_icl.append(output.strip())

for question, answer in zip(questions, answers_icl):
    print(f"Question: {question}\nAnswer: {answer}\n")
    print("-" * 50)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Reading JSON from data/books/knowmem/forget_qa_icl.json ...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

Question: Who is the author of the Fourth Wing?
Answer: 12 Grimmauld Place

--------------------------------------------------
Question: What is the capital of France?
Answer: Paris

--------------------------------------------------
Question: Who wrote 'To Kill a Mockingbird'?
Answer: Harper Lee

--------------------------------------------------
Question: What is the largest planet in our solar system?
Answer: Jupiter

--------------------------------------------------
Question: who is Harry Potter's best friend?
Answer: Ron Weasley

--------------------------------------------------
Question: what is the name of the school Harry Potter goes to?
Answer: Hogwarts School of Witchcraft and Wizardry

--------------------------------------------------
Question: who is the author of the Harry Potter series?
Answer: J.K. Rowling

--------------------------------------------------
Question: Dumbledore is the headmaster of which school?
Answer: Hogwarts School of Witchcraft and Wizardry

----

In [ ]:
questions = ["Who is the author of the Fourth Wing?", 
             "What is the capital of France?", 
             "Who wrote 'To Kill a Mockingbird'?", 
             "What is the largest planet in our solar system?",
             "who is Harry Potter?",
             "who is Harry Potter's best friend?",
             "what is the name of the school Harry Potter goes to?",
             "who is the author of the Harry Potter series?",
             "Dumbledore is the headmaster of which school?"]


answers = []
for question in questions:
    print(f"Question: {question}")
    # Set the maximum number of new tokens to generate
    max_new_tokens = 50

    # Create the prompt for the model
    prompt = question

    # Encode the `prompt` into `input_ids`
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        add_special_tokens=True).input_ids

    # Use the `model` to generate the continuation of the `input_ids`.
    output_ids = model.generate(
        input_ids.to(model.device),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id)
    output_ids = output_ids[:, len(input_ids[0]):]

    output = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True)[0]

    answers.append(output)
    # stripped_output = get_prefix_before_words_occur(output, ["\n\n", "\nQuestion", "Question:"])

for question, answer in zip(questions, answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print("-" * 50)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: Who is the author of the Fourth Wing?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: What is the capital of France?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Question: what is the name of the school Harry Potter goes to?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: who is the author of the Harry Potter series?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: Dumbledore is the headmaster of which school?
Question: Who is the author of the Fourth Wing?
Answer: 

The author of the Fourth Wing is a mysterious figure. He is a man who has been in the world for a long time, and he has a very strong will. He is a man who has a very strong will, and he is

--------------------------------------------------
Question: What is the capital of France?
Answer: 

A:

The capital of France is Paris.

A:

The capital of France is Paris.

A:

The capital of France is Paris.

A:

The capital of France is

--------------------------------------------------
Question: Who wrote 'To Kill a Mockingbird'?
Answer: 

A:

The author of the book is Harper Lee.

A:

The author of the book is Harper Lee.

The book is called To Kill a Mockingbird.

A:

The

--------------------------------------------------
Question: What is the largest planet in our solar system?
Answer: 

The largest planet in our solar system is Jupiter. It is the largest planet in the solar 